**Домашнее задание**

Обучите CNN (самописная) на CIFAR-100.

Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50.

*Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.

# **Подключение библиотек**



In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torchvision import models


from sklearn.model_selection import train_test_split

Подготовка данных. Загрузка датасета CIFAR-100

In [2]:
dataset = datasets.CIFAR100(root='./data/', train=True, download=True)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data/


In [3]:
class MyOwnCifar(Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        label = self._base_dataset[idx][1]
        return img, label

In [4]:
def train_valid_split(dataset):
    X_train, X_test = train_test_split(dataset, test_size=0.05, random_state=13)
    return X_train, X_test

In [5]:
train_dataset, test_dataset = train_valid_split(dataset)

In [6]:
train_dataset = MyOwnCifar(train_dataset, transforms.ToTensor())

In [7]:
test_dataset = MyOwnCifar(test_dataset, transforms.ToTensor())

In [8]:
train_loader = DataLoader(train_dataset,
                          batch_size=100,
                          shuffle=True, 
                          num_workers=0)

test_loader = DataLoader(test_dataset,
                          batch_size=100,
                          shuffle=False,
                          num_workers=0)

In [9]:
classes = dataset.classes
len(classes)

100

In [10]:
for image, label in train_dataset:
    print(f'image size: {image.shape}\nimage_tensor:\n{image}\nclass: {classes[label]}')
    break

image size: torch.Size([3, 32, 32])
image_tensor:
tensor([[[0.6039, 0.6118, 0.6275,  ..., 0.4824, 0.4863, 0.4824],
         [0.6353, 0.6431, 0.6510,  ..., 0.5059, 0.5176, 0.5059],
         [0.6510, 0.6549, 0.6667,  ..., 0.5216, 0.5333, 0.5216],
         ...,
         [0.4314, 0.4588, 0.4980,  ..., 0.4588, 0.4902, 0.4510],
         [0.4392, 0.4627, 0.4902,  ..., 0.4784, 0.4784, 0.4510],
         [0.4392, 0.4588, 0.4745,  ..., 0.4745, 0.4588, 0.4314]],

        [[0.6000, 0.6078, 0.6235,  ..., 0.4627, 0.4627, 0.4627],
         [0.6314, 0.6392, 0.6471,  ..., 0.4863, 0.4902, 0.4902],
         [0.6471, 0.6510, 0.6627,  ..., 0.5020, 0.5059, 0.5020],
         ...,
         [0.3843, 0.4157, 0.4510,  ..., 0.4118, 0.4431, 0.4039],
         [0.3882, 0.4157, 0.4431,  ..., 0.4314, 0.4314, 0.4039],
         [0.3922, 0.4118, 0.4275,  ..., 0.4275, 0.4118, 0.3843]],

        [[0.6314, 0.6392, 0.6510,  ..., 0.4863, 0.4824, 0.4824],
         [0.6627, 0.6706, 0.6784,  ..., 0.5137, 0.5098, 0.5059],
        

**Обучение самописной CNN на CIFAR-100.**

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm2d(3) # 3 input channels
        self.conv1 = nn.Conv2d(3, 30, 3) # convolute 3 channels to 25 channels by frame of 3*3
                
        self.bn2 = nn.BatchNorm2d(30) # batchnorm of 25 channels
        self.conv2 = nn.Conv2d(30, 60, 3) # convolute 25 channels to 75 channels by frame of 3*3
        
        self.bn3 = nn.BatchNorm2d(60) # batchnorm of 75 channels
        self.conv3 = nn.Conv2d(60, 100, 2) # convolute 75 channels to 150 channels by frame of 2*2
        
        self.bn4 = nn.BatchNorm2d(100) # batchnorm of 150 channels
        self.conv4 = nn.Conv2d(100, 150, 2) # convolute 150 channels to 300 channels by frame of 2*2
        
        self.bn5 = nn.BatchNorm2d(150) # batchnorm of 300 channels
                
        self.dp1 = nn.Dropout(0.25) # dropout for 1 linear with 40% non-trainable params
        self.dp2 = nn.Dropout(0.2) # dropout for 2 linear with 30% non-trainable params
        self.dp3 = nn.Dropout(0.2) # dropout for 3 linear with 20% non-trainable params
        
        self.fc1 = nn.Linear(400, 250) # 1th linear layer, 1600 inputs -> 800 outputs
        self.fc2 = nn.Linear(450, 200) # 2th linear layer, 800 inputs -> 400 outputs
        self.fc3 = nn.Linear(400, 200) # 3th linear layer, 400 inputs -> 200 outputs
        self.out = nn.Linear(250, 100) # 4th linear layer, 200 inputs -> 100 outputs
            
    def forward(self, x):
        # 1 layer conv
        x = self.bn1(x)
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        # 2 layer conv
        x = self.bn2(x)
        x = self.conv2(x)
        x = F.leaky_relu(x)
        x = F.max_pool2d(x, 2)
        
        # 3 layer conv
        x = self.bn3(x)
        x = self.conv3(x)
        x = F.relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        # 4 layer conv
        x = self.bn4(x)
                        
        # flatten
        x = x.view(x.size(0), -1) # transform x to vector
        
        # 5 layer linear
        x = self.dp1(x) 
        x = self.fc1(x)
        x = F.relu(x)
        
               
        return self.out(x)
       
net = Net()
net.cuda()
print(net)

Net(
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(60, 100, kernel_size=(2, 2), stride=(1, 1))
  (bn4): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(100, 150, kernel_size=(2, 2), stride=(1, 1))
  (bn5): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp1): Dropout(p=0.25, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
  (dp3): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=400, out_features=250, bias=True)
  (fc2): Linear(in_features=450, out_features=200, bias=True)
  (fc3): Linear(in_features=400, out_features=200, bias=True)

In [12]:
optimizer = Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [13]:
num_epochs = 10
net.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].cuda(), data[1].cuda()

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
            
                test_outputs = net(data[0].cuda())
                test_running_total += len(data[1])
                test_running_right += (data[1].cuda() == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/475]. Loss: 0.047. Acc: 0.010. Test acc: 0.007
Epoch [1/10]. Step [301/475]. Loss: 0.038. Acc: 0.122. Test acc: 0.160
Epoch [2/10]. Step [1/475]. Loss: 0.033. Acc: 0.160. Test acc: 0.204
Epoch [2/10]. Step [301/475]. Loss: 0.031. Acc: 0.231. Test acc: 0.247
Epoch [3/10]. Step [1/475]. Loss: 0.029. Acc: 0.280. Test acc: 0.262
Epoch [3/10]. Step [301/475]. Loss: 0.029. Acc: 0.285. Test acc: 0.283
Epoch [4/10]. Step [1/475]. Loss: 0.030. Acc: 0.320. Test acc: 0.297
Epoch [4/10]. Step [301/475]. Loss: 0.027. Acc: 0.317. Test acc: 0.298
Epoch [5/10]. Step [1/475]. Loss: 0.025. Acc: 0.370. Test acc: 0.313
Epoch [5/10]. Step [301/475]. Loss: 0.026. Acc: 0.344. Test acc: 0.330
Epoch [6/10]. Step [1/475]. Loss: 0.024. Acc: 0.420. Test acc: 0.340
Epoch [6/10]. Step [301/475]. Loss: 0.025. Acc: 0.363. Test acc: 0.338
Epoch [7/10]. Step [1/475]. Loss: 0.022. Acc: 0.420. Test acc: 0.348
Epoch [7/10]. Step [301/475]. Loss: 0.024. Acc: 0.384. Test acc: 0.359
Epoch [8/10]. Step [

**Обучение CNN на CIFAR-100 через дообучение ImageNet Resnet-50**

In [14]:
model = models.resnet50(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [15]:
for param in list(model.parameters())[:]:
    param.requires_grad = False

In [16]:
model.dp1 = nn.Dropout(0.2)
model.fc = nn.Linear(2048, 1024)
model.dp2 = nn.Dropout(0.2)
model.fc2 = nn.Linear(1024, 100)

In [17]:
trans_actions = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
                                  )
test_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
                                     )

train_dataset, test_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
test_dataset = MyOwnCifar(test_dataset, test_transforms)

train_loader = DataLoader(train_dataset,
                          batch_size=100,
                          shuffle=True,
                          num_workers=0)
test_loader = DataLoader(test_dataset,
                         batch_size=100,
                         shuffle=False,
                         num_workers=0)

In [18]:
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [19]:
model.cuda()
num_epochs = 10
model.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].cuda(), data[1].cuda()

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
            
                test_outputs = model(data[0].cuda())
                test_running_total += len(data[1])
                test_running_right += (data[1].cuda() == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        model.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/475]. Loss: 0.071. Acc: 0.000. Test acc: 0.017
Epoch [1/10]. Step [301/475]. Loss: 0.071. Acc: 0.154. Test acc: 0.182
Epoch [2/10]. Step [1/475]. Loss: 0.067. Acc: 0.240. Test acc: 0.198
Epoch [2/10]. Step [301/475]. Loss: 0.070. Acc: 0.222. Test acc: 0.188
Epoch [3/10]. Step [1/475]. Loss: 0.059. Acc: 0.310. Test acc: 0.205
Epoch [3/10]. Step [301/475]. Loss: 0.069. Acc: 0.248. Test acc: 0.202
Epoch [4/10]. Step [1/475]. Loss: 0.071. Acc: 0.300. Test acc: 0.206
Epoch [4/10]. Step [301/475]. Loss: 0.067. Acc: 0.268. Test acc: 0.222
Epoch [5/10]. Step [1/475]. Loss: 0.068. Acc: 0.260. Test acc: 0.212
Epoch [5/10]. Step [301/475]. Loss: 0.067. Acc: 0.279. Test acc: 0.228
Epoch [6/10]. Step [1/475]. Loss: 0.051. Acc: 0.470. Test acc: 0.229
Epoch [6/10]. Step [301/475]. Loss: 0.067. Acc: 0.292. Test acc: 0.224
Epoch [7/10]. Step [1/475]. Loss: 0.056. Acc: 0.440. Test acc: 0.217
Epoch [7/10]. Step [301/475]. Loss: 0.066. Acc: 0.307. Test acc: 0.237
Epoch [8/10]. Step [

ImageNet Resnet-50 показывает метрику на тесте ниже (0.24), чем самописная